# Lecture 3

# Exercise 5: Nested Sampling

As usual, we start with some setup:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


We will also use a new library here, called UltraNest, for nested sampling, healpy, and some tools to support it.

In [ ]:
import ultranest

Our goal will be to compare how well two models fit the data, LambdaCDM, where dark energy is a cosmological constant, and wCDM, where it can vary with a fixed equation of state $w(z) \equiv - p / \rho c^2 = w_0$.

Astropy has a model class for each of these.

In [ ]:
from astropy.cosmology import LambdaCDM, wCDM

We will use the same Pantheon data as before:

In [ ]:
# Load in the data
z_obs, m_obs = np.loadtxt("./data.txt").T
C = np.loadtxt("./cov.txt")
invC = np.linalg.inv(C)

Last exercise you wrote functions to compute the theory model and liklihood of the Pantheon data under the Lambda CDM model.
Copy those in here, but rename them `model_lcdm` and `loglike_lcdm`.

If you didn't complete the exercise, you can find model answers in the Lecture 2 Answers notebook.

In [ ]:
# Copy your functions in here.


We will be comparing to the wCDM model, so will need equivalent `model_wcdm` and `loglike_wcdm` functions to match these.
Write these functions here, remembering that we have a new parameter `w0` that needs to be included throughout

In [ ]:
# Complete these functions

def model_wcdm(H0, Omega_matter, Omega_lambda, M0, w0, z):
    ...
    return m


def loglike_wcdm(p):
    ...
    return L

Since nested sampling gradually moves in from the edges of the prior space, we need to specify what those priors
are and how to transform from the *unit cube*, to parameter values.

This means converting a number between 0 and 1 for each parameter to the full parameter space.  In general it would use the cumulative density function of the prior on each parameter, but to keep things simple we'll use uniform priors here, which makes everything much easier.

In this case, for each parameter we transform $u_i$ which is between 0 and 1 to $x_i$ which is between some minimum and maximum we specify, using $x_i = \mathrm{min}_i + (\mathrm{max}_i - \mathrm{min}_i) \cdot u_i$

HINT: You can use numpy's vector addition.

In [ ]:
# These are the min and max values we will use for each parameters.  What is the physical motivation behind each?
mins_lcdm = np.array([60., 0., 0., -20.])
maxs_lcdm = np.array([80., 1., 1., -18.])
mins_wcdm = np.array([60., 0., 0., -20., -2.])
maxs_wcdm = np.array([80., 1., 1., -18., -0.5])

# Complete thes functions
def prior_transform_wcdm(u):
    x = ...
    return x

def prior_transform_lcdm(u):
    x = ...
    return x


Now we can run our Nested Sampler.  We will use the most basic sampler that UltraNest offers - it also has more advanced versions that can be faster.

First we will do this for LambdaCDM.  It may take a minute or two to complete.

In [ ]:
# Run this code
param_names_lcdm = ['H0', 'Omega_matter', 'Omega_lambda', 'M0']
sampler_lcdm = ultranest.NestedSampler(param_names_lcdm, loglike_lcdm, prior_transform_lcdm, log_dir="./lcdm", num_live_points=100)
result_lcdm = sampler_lcdm.run()

Use the `print_results` method on the `sampler_lcdm` object to see the log of the estimated evidence Z:

In [ ]:
# Write your code here


Now we'll do the same for wCDM - write equivalent code for that model here and run it.

It will take a bit longer as it has an additional parameter.

In [ ]:
# Complete this code

param_names_wcdm = ...
sampler_wcdm = ...
result_wcdm = ...


Again, use the print_summary method on the sampler objects to print out log(Z) values


In [ ]:
# Write your code here


The log(Z) values are stored in the dictionaries result_lcdm and result_wcdm.

Extract them and compute the difference in log(Z) and the relative probability of the two models.  What can you conclude?

What is the error on this number (the dictionaries also contain 'logzerr')

In [ ]:
# Write your code here


# Exercise 6: Healpix

We'll now use the library `healpy` to explore some Healpix maps of the CMB sky made from Planck Space Telescope data.

In [ ]:
import healpy

Use the function `healpy.read_map` to load the Healpix file `"planck_cmb.fits"`.

This might generate a warning, which you can ignore.

In [ ]:
# Complete this code
m = ...

The function above should also print the nside parameter, defining the resolution of the map.
Define it here so we can use it later.

In [ ]:
# Complete this code
nside = ...

Use the function `healpy.mollview` to display the map:

In [ ]:
# Write your code here


Uh oh!  The core of th galaxy is so bright that it outshines everything else in the map, so the color scale makes the map look the same all over (the CMB fluctuations are very small).

Explore values to use for the `min` and `max` options of `mollview` to make the fluctuations in the map visible clearly.

In [ ]:
# Write your code here


Find the coordinates of the brightest point in the map, first by finding its pixel index and then by converting to sky coordinates.

HINT: `np.argmax` finds the index of the maximum value of an array.  `healpy.pix2ang` can convert to coordinates.  

In [ ]:
# Write your code here


We will now measure the power spectrum of the map, which should roughly trace this famous plot:
    https://lambda.gsfc.nasa.gov/product/map/dr2/map_images/PowerSpectrum512.png
    
There are seveal issues with the map, so we won't get the result straight away.
        
Use `healpy.anafast` to compute the power spectrum `c_ell` of the map.

In [ ]:
# Write your code here


Now plot it in the form `ell * (ell + 1) * c_ell / (2 * np.pi)`

In [ ]:
# Complete this code
ell = np.arange(cl.size)
f = ell*(ell+1)/2/np.pi

...

This doesn't look right yet! The bright core of the galaxy is the biggest contaminant.  Use the `gal_cut` argument to `anafast` to remove that region and plot it agan.  Have a look at the map to decide how big an area to remove.

In [ ]:
# Write your code here


That should be a little better, but there is still a noise level of about 2.1e-14 in the power spectrum `c_ell` .  Plot a version with that subtracted.

HINT: Note that it's subtracted from `c_ell`, not `ell * (ell + 1) * c_ell / (2 * np.pi)`.


In [ ]:
# Write your code here


At this Nside parameter the results of a transform are not very accurate above `2*Nside`.  Make a plot going up to just that ell value.

In [ ]:
# Write your code here


This should look at least vageuly like the plot above now!

## Further exercises

- Run the nested sampling with more live points and compare results
- Investigate `ultranest.ReactiveNestedSampler` and see how it behaves differently.
- Use `healpy.ud_grade` to transform the map to one of lower resolution, and plot the change to the `C_ell` values.
- Use camb to generate a CMB power spectrum (take the first TT part), and transform it into a map using healpy.
